In [248]:
import pandas as pd
import os
import warnings
import numpy as np
import re
import json

from functools import reduce
from datetime import datetime
from datetime import date

warnings.filterwarnings("ignore")

### ========================================================

In [249]:
test_0909_uni_test = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 
                'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
                'productionDate', 'vehicleTransmission', 'vendor', 
                'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']
test_0909_uni_0909 = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 
                 'Комплектация','fuelType', 'mileage', 'modelDate', 'model', 'numberOfDoors', 
                'productionDate', 'vehicleTransmission', 
                'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'price']

### ========================================================

In [250]:
df_full= pd.read_csv('./Preproject_7_data/' + 'all_auto_ru_09_09_2020.csv',low_memory=False)

In [251]:
test = pd.read_csv('./Project_7_data/test.csv',low_memory=False)
df= pd.read_csv('./Project_7_data/' + 'auto_ru_2020_09_09.csv',low_memory=False)

### ========================================================

In [252]:
def bodyType_unification(x):
    res=[]
    x = x.lower() if type(x) == str else x  # <================
    try:
        for body_type in test.bodyType.unique():
            if body_type in x:
                res.append(body_type)
    except Exception: return x
    if not res: 
        return x.split()[0]
    return max(res)

def engineDisplacement_unification(x):
    x = float(re.sub("[^\d.]", r'', x)) if re.sub("[^\d.]", r'', x) else 0
    if x >= 7: x = 0
    return x

def train_options(x):
    point = "'available_options': "
    start = x.find(point)+len(point)+2
    finish = x.find("]",start) - 1
    return x[start:finish].split("', '")

def test_ownership_unification(x):
    try:
        digits = re.findall('\d+',x) 
        if len(digits) == 2: res = int(digits[0])*12 + int(digits[1])
        elif len(digits) == 1 and 'месяц' in x: res = int(digits[0])
        elif len(digits) == 1 and 'месяц' not in x: res = int(digits[0])*12
    except Exception: res = 0   
    return res

def train_ownership_unification(x):
    tmp = json.loads(x.replace("'",'"'))  if x==x else {'year': 2020, 'month': 9}
    res = (2020 - tmp['year'])*12 + tmp['month'] - 9
    if res<0: res = 0
    return res


In [253]:
def test_unification(df_to_proc):
    test = df_to_proc.copy()[test_0909_uni_test]
    owner_dict = {'3 или более': 3., '2\xa0владельца': 2.,'1\xa0владелец': 1.}
    # engineDisplacement
    test.engineDisplacement = test.engineDisplacement.apply(
        lambda x: float(x[:-4]) if x[:-4] else np.NaN) 
    # enginePower
    test.enginePower = test.enginePower.apply(lambda x: int(x[:-4]))
    # equipment_dict
    test.equipment_dict = test.equipment_dict.apply(
        lambda x: list(json.loads(x).keys()) if x==x else [])
    # Владельцы
    test.Владельцы = test.Владельцы.map(owner_dict)
    # Владение
    test.Владение = test.Владение.apply(test_ownership_unification)
    return test

In [254]:
def train_0909_unification(df_to_proc):
    df = df_to_proc.copy()[test_0909_uni_0909]
    color_codes = {'040001': 'чёрный','FAFBFB': 'белый', '0000CC': 'синий', 
                   '200204': 'коричневый', 'EE1D19': 'красный', 'CACECB': 'серый',
                   'C49648': 'бежевый', '97948F': 'серебристый', 'FFD600': 'жёлтый',
                   'FF8649': 'оранжевый', '22A0F8': 'голубой','FFC0CB': 'розовый', 
                   'DEA522': 'золотистый', '007F00': 'зелёный', '660099': 'пурпурный',
                   '4A2197': 'фиолетовый'}
    transmission_dict = {'MECHANICAL':'механическая', 'AUTOMATIC':'автоматическая', 
                         'ROBOT':'роботизированная','VARIATOR':'вариатор'}
    vendor_dict = {'AUDI':'EUROPEAN','BMW':'EUROPEAN','HONDA':'JAPANESE','INFINITI':'JAPANESE',
                   'LEXUS':'JAPANESE','MERCEDES':'EUROPEAN','MITSUBISHI':'JAPANESE',
                   'NISSAN':'JAPANESE','SKODA':'EUROPEAN','TOYOTA':'JAPANESE',
                   'VOLKSWAGEN':'EUROPEAN','VOLVO':'EUROPEAN'}
    PTS_dict = {'ORIGINAL': 'Оригинал', 'DUPLICATE': 'Дубликат'}
    wheel_dict = {'LEFT':'Левый', 'RIGHT':'Правый'}
    # bodyType
    df.dropna(subset=['bodyType'],inplace=True)
    df.bodyType = df.bodyType.apply(bodyType_unification)
    # color
    df.color = df.color.map(color_codes)
    # engineDisplacement
    df.engineDisplacement = df.engineDisplacement.apply(engineDisplacement_unification)
    # enginePower
    df.enginePower = df.enginePower.astype(int)
    # equipment_dict
    df['equipment_dict'] = df.Комплектация.apply(train_options)
    df.drop(columns=['Комплектация'],inplace=True)
    # modelDate 
    df.modelDate = df.modelDate.astype(int)
    # model_name
    df['model_name'] = df.model
    df.drop(columns=['model'],inplace=True)
    # numberOfDoors
    df.numberOfDoors = df.numberOfDoors.astype(int)
    # vehicleTransmission
    df.vehicleTransmission = df.vehicleTransmission.map(transmission_dict)
    # vendor
    df['vendor'] = df.brand.map(vendor_dict)
    # Владение
    df.Владение = df.Владение.apply(train_ownership_unification)
    # ПТС
    df.ПТС = df.ПТС.map(PTS_dict)
    # Руль
    df.Руль = df.Руль.map(wheel_dict)
    
    return df

In [255]:
%%time 
test_unification(test)

Wall time: 1.13 s


,bodyType,brand,color,engineDisplacement,enginePower,equipment_dict,fuelType,mileage,modelDate,model_name,numberOfDoors,productionDate,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль
0,лифтбек,SKODA,синий,1.2,105,"[engine-proof, tinted-glass, airbag-driver, au...",бензин,74000,2013,OCTAVIA,5,2014,роботизированная,EUROPEAN,3.0,0,Оригинал,передний,Левый
1,лифтбек,SKODA,чёрный,1.6,110,"[cruise-control, asr, esp, airbag-driver, isof...",бензин,60563,2017,OCTAVIA,5,2017,механическая,EUROPEAN,1.0,0,Оригинал,передний,Левый
2,лифтбек,SKODA,серый,1.8,152,"[cruise-control, tinted-glass, esp, adaptive-l...",бензин,88000,2013,SUPERB,5,2014,роботизированная,EUROPEAN,1.0,0,Оригинал,передний,Левый
3,лифтбек,SKODA,коричневый,1.6,110,"[cruise-control, roller-blind-for-rear-window,...",бензин,95000,2013,OCTAVIA,5,2014,автоматическая,EUROPEAN,1.0,0,Оригинал,передний,Левый
4,лифтбек,SKODA,белый,1.8,152,"[cruise-control, asr, esp, airbag-driver, isof...",бензин,58536,2008,OCTAVIA,5,2012,автоматическая,EUROPEAN,1.0,0,Оригинал,передний,Левый
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34681,седан,BMW,коричневый,1.6,136,"[cruise-control, asr, tinted-glass, esp, start...",бензин,115000,2011,3ER,4,2014,автоматическая,EUROPEAN,3.0,25,Оригинал,задний,Левый
34682,седан,BMW,чёрный,2.0,190,[],дизель,98000,2016,5ER,4,2018,автоматическая,EUROPEAN,1.0,0,Оригинал,полный,Левый
34683,седан,BMW,серый,2.5,170,[],бензин,360000,1995,5ER,4,1997,автоматическая,EUROPEAN,3.0,0,Дубликат,задний,Левый
34684,внедорожник 5 дв.,BMW,коричневый,2.0,184,"[engine-proof, esp, start-stop-function, airba...",дизель,90500,2012,X1,5,2013,автоматическая,EUROPEAN,2.0,0,Оригинал,полный,Левый


In [256]:
%%time
train_0909_unification(df)

Wall time: 1min 33s


,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,modelDate,numberOfDoors,productionDate,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,price,equipment_dict,model_name,vendor
0,седан,AUDI,чёрный,2.8,174,бензин,350000,1990,4,1991,механическая,3.0,0,Оригинал,полный,Левый,200000.0,[],100,EUROPEAN
1,седан,AUDI,красный,1.8,90,бензин,173424,1982,4,1986,механическая,3.0,0,Оригинал,передний,Левый,60000.0,[],100,EUROPEAN
2,универсал 5 дв.,AUDI,синий,2.3,136,бензин,230000,1988,5,1989,механическая,3.0,0,Оригинал,полный,Левый,99000.0,[],100,EUROPEAN
3,седан,AUDI,серый,1.8,90,бензин,240000,1988,4,1989,механическая,3.0,0,Оригинал,передний,Левый,65000.0,[],100,EUROPEAN
4,седан,AUDI,чёрный,2.0,101,бензин,300000,1990,4,1991,механическая,3.0,0,Дубликат,передний,Левый,100000.0,[],100,EUROPEAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49304,купе,MERCEDES,золотистый,3.0,150,бензин,45200,1951,2,1952,механическая,1.0,195,Оригинал,задний,Левый,10800000.0,[],W188,EUROPEAN
49305,седан,MERCEDES,чёрный,3.0,160,бензин,10000,1957,4,1959,автоматическая,2.0,0,Оригинал,задний,Левый,5600000.0,[],W189,EUROPEAN
49306,пикап двойная кабина,MERCEDES,серебристый,0.0,258,дизель,800,2017,4,2018,автоматическая,1.0,15,Оригинал,полный,Левый,4000000.0,[],X_KLASSE,EUROPEAN
49307,пикап двойная кабина,MERCEDES,серебристый,0.0,190,дизель,15850,2017,4,2018,автоматическая,1.0,0,Оригинал,полный,Левый,3150000.0,"[cruise-control, heated-wash-system, airbag-pa...",X_KLASSE,EUROPEAN


In [257]:
%%time
train_0909_unification(df_full)

Wall time: 2min 52s


,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,modelDate,numberOfDoors,productionDate,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,price,equipment_dict,model_name,vendor
0,седан,AUDI,чёрный,2.8,174,бензин,350000,1990,4,1991,механическая,3.0,0,Оригинал,полный,Левый,200000.0,[],100,EUROPEAN
1,седан,AUDI,красный,1.8,90,бензин,173424,1982,4,1986,механическая,3.0,0,Оригинал,передний,Левый,60000.0,[],100,EUROPEAN
2,универсал 5 дв.,AUDI,синий,2.3,136,бензин,230000,1988,5,1989,механическая,3.0,0,Оригинал,полный,Левый,99000.0,[],100,EUROPEAN
3,седан,AUDI,серый,1.8,90,бензин,240000,1988,4,1989,механическая,3.0,0,Оригинал,передний,Левый,65000.0,[],100,EUROPEAN
4,седан,AUDI,чёрный,2.0,101,бензин,300000,1990,4,1991,механическая,3.0,0,Дубликат,передний,Левый,100000.0,[],100,EUROPEAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89373,минивэн,SSANG_YONG,чёрный,2.7,163,дизель,260000,2004,5,2006,автоматическая,3.0,0,Оригинал,полный,Левый,450000.0,[],RODIUS,NaN
89374,минивэн,SSANG_YONG,серый,2.7,163,дизель,300000,2004,5,2007,автоматическая,3.0,29,Оригинал,полный,Левый,485000.0,[],RODIUS,NaN
89375,минивэн,SSANG_YONG,чёрный,2.0,149,дизель,126000,2013,5,2014,автоматическая,1.0,63,Оригинал,полный,Левый,1090000.0,"[cruise-control, multi-wheel, airbag-passenger...",STAVIC,NaN
89376,минивэн,SSANG_YONG,серебристый,2.0,149,дизель,84000,2013,5,2013,автоматическая,3.0,37,Оригинал,полный,Левый,1180000.0,"[cruise-control, multi-wheel, airbag-passenger...",STAVIC,NaN


## =================PARS===========================

In [258]:
pars = pd.read_csv('./Project_7_data/train_new.csv')

In [259]:
[print (x) for x in set(pars.columns) & set(test.columns)];

car_url
engineDisplacement
vehicleTransmission
Руль
sell_id
modelDate
color
Привод
productionDate
numberOfDoors
vendor
Владельцы
mileage
enginePower
model_name
ПТС
bodyType
equipment_dict
brand


#### car_url

In [260]:
print(test.car_url[5])

https://auto.ru/cars/used/sale/skoda/octavia_rs/1100912634-456da88e/


In [261]:
print(pars.car_url[5])

https://auto.ru/cars/used/sale/skoda/fabia/1114894898-c8987686/


#### engineDisplacement

In [262]:
test.engineDisplacement.apply(
    lambda x: float(x[:-4]) if x[:-4] else np.NaN).unique()

array([1.2, 1.6, 1.8, 2. , 1.4, 1.3, 1. , 3.6, 1.5, 1.9, 2.8, 1.1, 2.5,
       4.2, 3. , 4. , 5.9, 2.7, 3.1, 2.4, 5.2, 3.2, 4.1, 6.3, 2.3, 6. ,
       2.2, 3.7, 2.9, 5. , 3.3, 2.1, 2.6, nan, 3.5, 1.7, 0.7, 4.4, 4.8,
       5.4, 6.6, 4.9, 3.8, 3.4, 3.9, 4.6, 5.6, 4.5, 5.5, 6.2, 4.7, 4.3,
       5.8, 5.3, 5.7])

In [263]:
pars.engineDisplacement.unique()

array([1.4, 1.2, 1.6, nan, 1.3, 2. , 1.5, 1.8, 2.8, 1.9, 3.6, 3. , 2.5,
       2.2, 3.5, 4.4, 2.7, 0.6, 4.8, 3.4, 6. , 5. , 5.4, 4. , 2.9, 6.6,
       0.7, 3.3, 4.9, 4.6, 3.2, 2.4, 2.3, 1.7, 3.7, 4.5, 5.6, 2.1, 2.6,
       6.2, 5.5, 5.8, 4.7, 4.2, 4.3, 3.8, 1. , 5.7, 4.1, 3.1, 5.9, 5.2])

#### vehicleTransmission

In [264]:
test.vehicleTransmission.unique() ,pars.vehicleTransmission.unique()

(array(['роботизированная', 'механическая', 'автоматическая', 'вариатор'],
       dtype=object),
 array(['механическая', 'роботизированная', 'автоматическая', nan,
        'вариатор'], dtype=object))

#### Руль

In [265]:
test.Руль.unique(),pars.Руль.unique()

(array(['Левый', 'Правый'], dtype=object),
 array(['Левый', nan, 'Правый'], dtype=object))

#### sell_id

In [266]:
test.sell_id[0],pars.sell_id[0]

(1100575026, 1114778882)

#### modelDate

In [267]:
test.modelDate = test.modelDate.astype(float)
test.modelDate.unique()

array([2013., 2017., 2008., 2009., 2016., 2012., 2015., 2010., 2006.,
       2000., 2007., 1994., 2004., 1999., 2005., 1976., 2001., 1969.,
       1996., 1998., 1989., 1934., 2014., 2011., 2018., 1986., 1997.,
       1990., 2019., 2002., 1991., 1987., 1980., 1982., 1938., 1988.,
       2003., 1983., 1978., 1979., 1984., 1992., 1995., 1993., 1985.,
       1974., 1966., 1977., 1981., 1972., 1968., 1975., 1949., 1937.,
       1936., 1973., 1959., 1958., 2020., 1965., 1971., 1904., 1963.,
       1955., 1951., 1960.])

In [268]:
model_generation_year = pd.read_csv(".\Project_7_data\model_generation_year.csv")
model_generation_year = dict(zip(model_generation_year.full_name + ' ' + model_generation_year.bodytype,
         model_generation_year.generation_year))
pars.modelDate = (pars.modelDate + ' ' + pars.bodyType).map(model_generation_year)

pars.modelDate.unique()

array([2010., 2005., 2007.,   nan, 2001., 2000., 2002., 2011., 1995.,
       1996., 1998., 1994., 2020., 2017., 2013., 2008., 2014., 2018.,
       2015., 2006., 2004., 2019., 2012., 1939., 1949., 1991., 1999.,
       1992., 1985., 2016., 1977., 2021., 1983., 2003., 1988., 1987.,
       1986., 2009., 1993., 1990., 1989., 1997., 1960., 1966., 1982.,
       1980., 1984., 1979., 1978., 1967., 1981., 1972.])

#### color

In [269]:
set(pars.color.unique()) - set(test.color.unique())

{nan}

#### Привод

In [270]:
test.Привод.unique(),pars.Привод.unique()

(array(['передний', 'полный', 'задний'], dtype=object),
 array(['передний', nan, 'полный', 'задний'], dtype=object))

#### productionDate

In [271]:
test.productionDate = test.productionDate.astype(float)
test.productionDate[:5]

0    2014.0
1    2017.0
2    2014.0
3    2014.0
4    2012.0
Name: productionDate, dtype: float64

In [272]:
pars.productionDate[:5]

0    2014.0
1    2012.0
2    2014.0
3    2012.0
4    2013.0
Name: productionDate, dtype: float64

#### numberOfDoors

In [273]:
test.numberOfDoors = test.numberOfDoors.astype(float)
test.numberOfDoors.unique()

array([5., 4., 2., 3., 0.])

In [274]:
pars.numberOfDoors.unique()

array([ 5., nan,  3.,  2.])

#### vendor

In [275]:
test.vendor.unique(), pars.vendor.unique()

(array(['EUROPEAN', 'JAPANESE'], dtype=object),
 array(['EUROPEAN', 'NAN', 'JAPANESE'], dtype=object))

#### Владельцы

In [276]:
owner_dict = {
    '3 или более': 3., 
    '2\xa0владельца': 2.,
    '1\xa0владелец': 1.
}
test.Владельцы = test.Владельцы.map(owner_dict)
pars.Владельцы = pars.Владельцы.map(owner_dict)

In [277]:
test.Владельцы.unique(), pars.Владельцы.unique()

(array([3., 1., 2.]), array([ 2.,  3.,  1., nan]))

#### mileage

In [278]:
test.mileage = test.mileage.astype(float)
test.mileage.unique()

array([ 74000.,  60563.,  88000., ..., 121276., 212678., 157965.])

In [279]:
pars.mileage.unique()

array([121000., 108980., 122000., ..., 413708., 410361., 255580.])

#### enginePower

In [280]:
test.enginePower = test.enginePower.apply(lambda x: int(x[:-4])).astype(float)
test.enginePower.unique()[:5]

array([105., 110., 152., 200., 102., 150.,  90., 180., 220., 122.,  70.,
       140., 125.,  54.,  86.,  75.,  64.,  95., 260., 170.,  80.,  68.,
       160., 115., 280.,  53.,  60., 143.,  42., 101.,  58., 193.,  79.,
        30., 100.,  50., 163., 225., 420., 211., 245., 560., 500., 249.,
       450., 605., 250., 354., 120., 290., 230., 350., 204., 255., 340.,
       177., 272., 372., 210., 130., 300., 190., 239., 435., 333., 271.,
       326., 238., 310., 233., 252., 133., 460., 520., 400., 525., 367.,
       265., 550., 580.,  88., 165., 430., 335., 174., 224., 136., 256.,
       338., 112., 218., 360.,  93., 182., 113., 240., 270., 137., 155.,
        61., 254., 116.,  71., 285., 408., 166., 156., 257., 201., 162.,
       194., 142., 154., 147., 281., 243., 118., 128.,  91.,  83., 205.,
       247., 188., 295., 275.,  78., 109.,  94., 185., 258., 145., 132.,
       135.,  52.,  66., 124.,  98., 215., 158.,  99., 103., 131., 126.,
       235., 181., 315., 203., 231.,  82., 304., 17

In [281]:
pars.enginePower.unique()[:5]

array([ 86., 105.,  70.,  54.,  60.,  nan,  75.,  68.,  64., 101., 180.,
        58., 150., 110., 125., 190., 102., 140., 152., 122., 160., 143.,
        80., 200., 220.,  90., 193., 170., 260., 136., 177., 115., 265.,
       129., 156., 306., 218., 163., 320., 204., 184., 116., 326.,  95.,
        46.,  51., 245., 258., 192., 249., 113., 374., 272., 387., 231.,
       118., 100., 252., 340., 235., 530., 300., 462., 407.,  19., 450.,
       313., 333., 367., 238., 400., 211., 544., 381., 286., 185., 445.,
       610., 609., 197., 188., 410., 370., 420., 510., 431., 343., 460.,
       560., 625., 600., 507., 315., 635., 248., 360., 347., 355., 264.,
       269., 575., 555., 321., 212., 201., 155., 154., 240., 145., 132.,
        53., 158., 142., 174.,  91., 130., 120.,  71., 109., 169., 166.,
       186., 128., 147., 162., 112., 275., 271., 194., 281.,  88., 131.,
       124.,  83.,  78., 205., 295.,  52.,  66., 257., 243., 133., 135.,
       250., 247., 237., 302., 222., 307., 280., 32

#### model_name

In [285]:
test.model_name.unique()[:5]

array(['OCTAVIA', 'SUPERB', 'OCTAVIA_RS', 'YETI', 'KODIAQ'], dtype=object)

In [284]:
pars.model_name.unique()[:5]

array(['Fabia', nan, 'Fabia RS', 'Felicia', 'Forman'], dtype=object)

#### ПТС

In [289]:
test.ПТС.unique(), pars.ПТС.unique()

(array(['Оригинал', 'Дубликат', nan], dtype=object),
 array(['Оригинал', 'Дубликат', nan], dtype=object))

#### bodyType

In [294]:
set(test.bodyType.unique()) - set(pars.bodyType.unique())

{'фастбек'}

#### equipment_dict

In [298]:
test.equipment_dict = test.equipment_dict.apply(lambda x: list(json.loads(x).keys()) if x==x else [])

In [340]:
pars.equipment_dict = pars.equipment_dict[:50].apply(
    lambda x: pd.Series(list(json.loads(x.replace("'",'"')).values())).sum()
    if x==x else x)

In [348]:
test.equipment_dict[0][:2],pars.equipment_dict[0][:2]

(['engine-proof', 'tinted-glass'],
 ['Подушка безопасности водителя', 'Подушка безопасности пассажира'])

#### 'description'

In [349]:
test.description[0]

'Все автомобили, представленные в продаже, проходят тщательную проверку по более 40 параметрам. Предоставляем гарантию юридической чистоты, а так же год технической гарантии на двигатель и КПП. Бесплатный тест-драйв. Возможно оформление автомобиля в кредит!\n\nПреимущества автокредитования:\n— Первоначальный взнос от 0%;\n— Более 30 кредитных программ;\n— Процентная ставка от 6% годовых;\n— Срок кредита от 6 месяцев до 7 лет;\n— Оформление кредита по двум документам;\n— Досрочное погашение без штрафов и комиссий;\n— Сумма кредита до 2 млн рублей;\n— Оформление КАСКО – по желанию;\n— Без справок и поручителей.\n\nСотрудничаем с 12 аккредитованными и сертифицированными банками РФ, среднее время ожидания решения банка 20–30 минут.\n\nПри покупке автомобиля по программе «Trade-in – выгодный обмен» или в кредит, получите дополнительную скидку до 80 000 рублей на данный автомобиль!\n\nЗаписаться на тест-драйв, а так же получить подробную информацию можно у специалистов автоцентра по указанно

In [350]:
# pars.description[0]

#### brand

In [351]:
test.brand.unique()

array(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'],
      dtype=object)

In [352]:
pars.brand.unique()

array(['SKODA', nan, 'BMW', 'HONDA', 'INFINITI', 'MERCEDES-BENZ',
       'NISSAN', 'LEXUS', 'TOYOTA', 'AUDI', 'VOLVO', 'VOLKSWAGEN',
       'MITSUBISHI'], dtype=object)

#### offerprice

In [355]:
pars.offerprice

0         470000.0
1         849000.0
2         470000.0
3         565000.0
4        1103000.0
           ...    
23216     320000.0
23217     269000.0
23218     190000.0
23219     420000.0
23220     110000.0
Name: offerprice, Length: 23221, dtype: float64

## ===================EDA===========================